In [1]:
import pandas as pd

In [2]:
df_lesson = pd.read_csv('lessons.txt', sep='|', skiprows=[1])
df_participants = pd.read_csv('participants.txt', sep='|', skiprows=[1])
df_quality = pd.read_csv('quality.txt', sep='|', skiprows=[1])
df_users = pd.read_csv('users.txt', sep='|', skiprows=[1])

In [3]:
df_lesson.columns = [name.strip() for name in df_lesson.columns.values.tolist()]
df_lesson['scheduled_time'] = pd.to_datetime(df_lesson['scheduled_time'])
df_lesson.dropna(inplace=True, subset=['event_id'])

df_lesson['id'] = df_lesson['id'].astype('str')
df_lesson['id'] = df_lesson['id'].str.strip()

df_lesson['subject'] = df_lesson['subject'].str.strip()
df_lesson['subject'] = df_lesson['subject'].astype('category')

df_lesson['event_id'] = df_lesson['event_id'].astype(int)

df_lesson = df_lesson[df_lesson['subject'] == 'phys'].drop(columns=['subject'])


In [4]:
df_participants.columns = [name.strip() for name in df_participants.columns.values.tolist()]
df_participants.dropna(inplace=True)

df_participants['event_id'] = df_participants['event_id'].astype('int')

df_participants['user_id'] = df_participants['user_id'].astype('str')
df_participants['user_id'] = df_participants['user_id'].str.strip()


In [5]:
columns = [name.strip() for name in df_quality.columns.values.tolist()]
df_quality.columns = columns
df_quality['lesson_id'] = df_quality['lesson_id'].astype('str')
df_quality['lesson_id'] = df_quality['lesson_id'].str.strip()
df_quality['tech_quality'] = pd.to_numeric(df_quality['tech_quality'], errors='coerce').fillna(0).astype(int)
grb = df_quality.groupby('lesson_id')
df_quality = grb['tech_quality'].sum().to_frame()
df_quality.reset_index(inplace=True)
df_quality = df_quality[1:]

In [6]:
df_users.columns = [name.strip() for name in df_users.columns.values.tolist()]
df_users.dropna(subset=['role'],inplace=True)
df_users['role'] = df_users['role'].str.strip()
df_users['id'] = df_users['id'].astype('str')
df_users['id'] = df_users['id'].str.strip()
df_users.drop_duplicates(subset ="id", inplace = True) 
df_users = df_users[df_users['role'] == 'tutor']

In [7]:
df_lesson = df_lesson.merge(df_participants,left_on='event_id', right_on='event_id', how='left').drop(columns=['event_id'])
df_lesson

,id,scheduled_time,user_id
0,1e7bb408-cfef-4a9f-8328-351c9483a64c,2020-01-19 12:00:00,e28351f5-4ccb-4549-8647-d43f2b15e7b8
1,1e7bb408-cfef-4a9f-8328-351c9483a64c,2020-01-19 12:00:00,4df2832a-1d63-4453-9659-43993fc35996
2,1e7bb408-cfef-4a9f-8328-351c9483a64c,2020-01-19 12:00:00,e28351f5-4ccb-4549-8647-d43f2b15e7b8
3,1e7bb408-cfef-4a9f-8328-351c9483a64c,2020-01-19 12:00:00,4df2832a-1d63-4453-9659-43993fc35996
4,00fc6685-f53a-49bb-b960-5e0042fd3852,2020-01-17 12:00:00,4ab072ed-8a41-4ed6-9877-8f1d1fbb24d2
...,...,...,...
482,457accf4-bd99-4b9d-b540-ef200e65afbd,2020-01-19 05:00:00,b1fe2c06-d284-4698-9f07-495c8f991700
483,403707fd-b110-4216-b678-6dbc92eb6fad,2020-01-19 11:00:00,0d688d4b-b5b7-4519-ab32-785138b3a330
484,403707fd-b110-4216-b678-6dbc92eb6fad,2020-01-19 11:00:00,8fe03f08-8581-430c-a590-9888ab36deb3
485,403707fd-b110-4216-b678-6dbc92eb6fad,2020-01-19 11:00:00,0d688d4b-b5b7-4519-ab32-785138b3a330


In [8]:
df_lesson = df_lesson.merge(df_quality,left_on='id', right_on='lesson_id', how='left').drop(columns=['lesson_id'])
names = df_lesson.columns.values.tolist()
names[names.index('id')] = 'lesson_id'
df_lesson.columns = names
df_lesson['tech_quality'] = df_lesson['tech_quality'].fillna(0)

In [9]:
df = df_lesson.merge(df_users,left_on='user_id', right_on='id').drop(columns=['id'])

In [10]:
del(df_lesson)
del(df_participants)
del(df_users)

In [11]:
df.drop_duplicates(subset =['lesson_id','user_id'], inplace = True)

In [12]:
df.head()

,lesson_id,scheduled_time,user_id,tech_quality,role
0,1e7bb408-cfef-4a9f-8328-351c9483a64c,2020-01-19 12:00:00,4df2832a-1d63-4453-9659-43993fc35996,0.0,tutor
2,0e30fdbd-2af7-40c7-8f38-336c9d8512f3,2020-01-11 12:00:00,4df2832a-1d63-4453-9659-43993fc35996,0.0,tutor
3,06277d4e-db4a-4fa0-b9f8-b1063088ad75,2020-01-20 14:00:00,4df2832a-1d63-4453-9659-43993fc35996,0.0,tutor
4,883e272f-c15b-4bf7-8066-f4a8e1862249,2020-01-12 09:30:00,4df2832a-1d63-4453-9659-43993fc35996,5.0,tutor
6,2e3b1173-7edb-4737-9500-a23d7b852518,2020-01-12 12:00:00,4df2832a-1d63-4453-9659-43993fc35996,5.0,tutor


Для решения задачки нужно сделать следующее:
1. Найти все уроки по физике (subject=phys).
2. В каждый день (начало и конец дня считается по московскому времени, то есть UTC+3:00) для каждого репетитора посчитать среднюю арифметическую оценку за его уроки (учитывать только уроки из п. 1). То есть, если учитель проводил в этот день три урока по физике, один из них он оценил на 3, а ученик оценил его на 4, второму уроку оценку поставил только ученик, и эта оценка 5, а третий урок вообще никто не оценил, то средняя арифметическая оценка учителя за уроки = (3 + 4 + 5) / 3 = 4.
3. Найти учителя, который в этот день имеет самую низкую среднюю арифметическую оценку за уроки (среди всех учителей, проводивших уроки по физике в этот день).
4. Вывести его в формате "<день> <id учителя> <средняя арифметическая оценка>". Оценку можно округлить с точностью до двух знаков после запятой. То есть как-то так:
2020-01-11 73c9af08-8581-430c-a590-9888ab36deb3 3.67
2020-01-12 909c2c8e-c054-4e9f-a51a-50bf5660f364 3.25

In [15]:
df2 = df.set_index(['scheduled_time']).groupby([pd.Grouper(freq='D'), 'user_id'])['tech_quality'].mean().round(2)

In [16]:
df2 

scheduled_time  user_id                             
2020-01-11      07bce6b2-ef5c-450d-9565-966c41260440     0.00
                2fa2ab62-f1b0-4036-872f-bcfd9a8686ff     5.00
                30a19496-bdaf-461c-abbc-2709ae520201     5.00
                43efce48-94b2-4412-857f-223d45969008     3.75
                4df2832a-1d63-4453-9659-43993fc35996     0.00
                                                        ...  
2020-01-20      696c838e-c054-4e9f-a51a-50bf5660f364     5.00
                b37ccae8-fc31-4ad8-8f55-ca855b23fbf6     8.33
                be676776-8366-4c71-8a35-d58014806eb5    10.00
                c0c073c8-cb9c-4ebc-ac13-472c06029591     0.00
                c6718d0e-976c-4d6c-b0e0-32c770776567     0.00
Name: tech_quality, Length: 99, dtype: float64